In [2]:
!pip install --quiet torch pymongo numpy pandas tqdm


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pymongo import MongoClient
import numpy as np, pandas as pd, torch, torch.nn as nn
from tqdm.auto import tqdm
client = MongoClient('mongodb://localhost:27017')
heroes_json=list(client.dota.heroes.find({}, {'_id':0}))
print('Heroes', len(heroes_json))


Heroes 126


In [7]:
PROFILE_COLS=['phase_1_winrate','phase_2_winrate','pro_winrate']
ROLE_LIST=['Carry','Mid','Support','Durable','Disabler','Nuker','Pusher','Initiator']
ROLE_IDX={r:i for i,r in enumerate(ROLE_LIST)}

hero_profile={}
role_matrix=np.zeros((max(h['id'] for h in heroes_json)+1, len(ROLE_LIST)),dtype=np.float32)
for doc in heroes_json:
    hid=doc['id']
    base=np.asarray([doc.get(c,0.5) for c in PROFILE_COLS],dtype=np.float32)
    role_vec=np.zeros(len(ROLE_LIST),dtype=np.float32)
    for r in doc.get('roles',[]): 
        if r in ROLE_IDX: role_vec[ROLE_IDX[r]]=1.
    hero_profile[hid]=np.concatenate([base, role_vec])
    role_matrix[hid]=role_vec
PROFILE_DIM=len(PROFILE_COLS)+len(ROLE_LIST)
HERO_COUNT=len(role_matrix)
print('PROFILE_DIM',PROFILE_DIM,'HERO_COUNT',HERO_COUNT)


PROFILE_DIM 11 HERO_COUNT 146


In [9]:
def agg_profile(ids):
    if not ids: return np.zeros(PROFILE_DIM,dtype=np.float32)
    return np.mean([hero_profile[i] for i in ids],axis=0)
def role_counts(ids):
    n_carry   = int(sum(role_matrix[i, ROLE_IDX['Carry']]   for i in ids))
    n_support = int(sum(role_matrix[i, ROLE_IDX['Support']] for i in ids))
    return n_carry, n_support


In [10]:
def int_idx(lst, offset=0):
    if not lst:
        return np.array([], dtype=np.int64)
    return offset + np.asarray(lst, dtype=np.int64)

matches=list(client.dota.matches_info.find({}, {'picks_bans':1}))
PHASES=[(2,2),(2,2),(1,1)]
examples=[]
for m in tqdm(matches):
    pb=m.get('picks_bans')
    if not pb or sum(1 for p in pb if p.get('is_pick'))!=10: continue
    picks=sorted([p for p in pb if p['is_pick']], key=lambda x:x['order'])
    rad=[p['hero_id'] for p in picks if p['team']==0]
    dire=[p['hero_id'] for p in picks if p['team']==1]
    r_idx=d_idx=0
    for r_n,d_n in PHASES:
        r_now=rad[r_idx:r_idx+r_n]; d_now=dire[d_idx:d_idx+d_n]
        for i,h in enumerate(r_now):
            my=rad[:r_idx+i]; opp=dire[:d_idx]
            c,s=role_counts(my)
            x=np.zeros(2*HERO_COUNT+2*PROFILE_DIM+2,dtype=np.float32)
            x[int_idx(my)] = 1.
            x[int_idx(opp, HERO_COUNT)] = 1.
            x[2*HERO_COUNT:2*HERO_COUNT+PROFILE_DIM]=agg_profile(my)
            x[2*HERO_COUNT+PROFILE_DIM:2*HERO_COUNT+2*PROFILE_DIM]=agg_profile(opp)
            x[-2]=c; x[-1]=s
            mask=np.zeros(HERO_COUNT,dtype=bool); mask[int_idx(my + opp)] = True
            examples.append((x,mask,h))
        for i,h in enumerate(d_now):
            my=dire[:d_idx+i]; opp=rad[:r_idx+r_n]
            c,s=role_counts(my)
            x=np.zeros(2*HERO_COUNT+2*PROFILE_DIM+2,dtype=np.float32)
            x[int_idx(opp)] = 1.
            x[int_idx(my, HERO_COUNT)] = 1.
            x[2*HERO_COUNT:2*HERO_COUNT+PROFILE_DIM]=agg_profile(opp)
            x[2*HERO_COUNT+PROFILE_DIM:2*HERO_COUNT+2*PROFILE_DIM]=agg_profile(my)
            x[-2]=c; x[-1]=s
            mask=np.zeros(HERO_COUNT,dtype=bool); mask[int_idx(my + opp)] = True
            examples.append((x,mask,h))
        r_idx+=r_n; d_idx+=d_n
print('Examples', len(examples))
INPUT_DIM=2*HERO_COUNT+2*PROFILE_DIM+2


100%|██████████| 44373/44373 [00:06<00:00, 6525.49it/s]

Examples 262867


In [7]:
class DraftDS(torch.utils.data.Dataset):
    def __init__(self,samples):
        self.x=[torch.from_numpy(s[0]) for s in samples]
        self.m=[torch.from_numpy(s[1]) for s in samples]
        self.y=torch.tensor([s[2] for s in samples],dtype=torch.long)
    def __len__(self): return len(self.y)
    def __getitem__(self,i): return self.x[i],self.m[i],self.y[i]

ds=DraftDS(examples)
train_size=int(0.9*len(ds)); val_size=len(ds)-train_size
train_ds, val_ds = torch.utils.data.random_split(ds,[train_size,val_size],
                                                generator=torch.Generator().manual_seed(42))
train_dl=torch.utils.data.DataLoader(train_ds,batch_size=1024,shuffle=True,num_workers=0,pin_memory=True)
val_dl  =torch.utils.data.DataLoader(val_ds,batch_size=1024,shuffle=False,num_workers=0,pin_memory=True)


In [6]:
class DraftNet(nn.Module):
    def __init__(self,input_dim):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(input_dim,512), nn.SiLU(),
            nn.Linear(512,256),       nn.SiLU(),
            nn.Linear(256,HERO_COUNT)
        )
    def forward(self,x,mask):
        logits=self.layers(x)
        return logits.masked_fill(mask.bool(), -65504.0)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model=DraftNet(INPUT_DIM).to(device)


NameError: name 'INPUT_DIM' is not defined

In [10]:
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler

EPOCHS   = 20
BATCH_SZ = train_dl.batch_size
scaler   = GradScaler()
loss_fn  = nn.CrossEntropyLoss()
opt      = torch.optim.AdamW(model.parameters(), lr=3e-4)

torch.backends.cudnn.benchmark = True

def top1(out, target):
    return (out.topk(1, -1).indices.squeeze(-1) == target).float().mean().item()

for ep in range(1, EPOCHS + 1):
    model.train()
    train_bar = tqdm(train_dl, desc=f"Epoch {ep:02d} [train]", leave=False)
    tl = 0
    for x, m, y in train_bar:
        x, m, y = x.to(device, non_blocking=True), m.to(device, non_blocking=True), y.to(device, non_blocking=True)

        opt.zero_grad(set_to_none=True)
        with autocast():
            out  = model(x, m)
            loss = loss_fn(out, y)
        scaler.scale(loss).backward()
        scaler.step(opt)
        scaler.update()

        tl += loss.item() * x.size(0)
        train_bar.set_postfix(loss=loss.item())

    model.eval()
    vl = va1 = 0
    with torch.no_grad(), autocast():
        for x, m, y in tqdm(val_dl, desc=f"Epoch {ep:02d} [valid]", leave=False):
            x, m, y = x.to(device, non_blocking=True), m.to(device, non_blocking=True), y.to(device, non_blocking=True)
            out  = model(x, m)
            loss = loss_fn(out, y)

            vl  += loss.item() * x.size(0)
            va1 += (out.topk(1, -1).indices.squeeze(-1) == y).float().sum().item()

    n_train, n_val = len(train_dl.dataset), len(val_dl.dataset)
    print(f"Ep{ep:02d} | train_loss {tl/n_train:.4f} | "
          f"val_loss {vl/n_val:.4f} | val@1 {va1/n_val:.3f}")


C:\Users\taras\AppData\Local\Temp\ipykernel_9720\1445708123.py:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler   = GradScaler()
Epoch 01 [train]:   0%|          | 0/232 [00:00<?, ?it/s]C:\Users\taras\AppData\Local\Temp\ipykernel_9720\1445708123.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\taras\AppData\Local\Temp\ipykernel_9720\1445708123.py:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), autocast():


Ep01 | train_loss 4.3526 | val_loss 4.3526 | val@1 0.057


Ep02 | train_loss 4.3438 | val_loss 4.3459 | val@1 0.059


Ep03 | train_loss 4.3362 | val_loss 4.3422 | val@1 0.058


Ep04 | train_loss 4.3305 | val_loss 4.3374 | val@1 0.058


...

Ep16 | train_loss 4.2975 | val_loss 4.3378 | val@1 0.057


Ep17 | train_loss 4.2953 | val_loss 4.3375 | val@1 0.056


Ep18 | train_loss 4.2933 | val_loss 4.3382 | val@1 0.057


Ep19 | train_loss 4.2911 | val_loss 4.3410 | val@1 0.056


Ep20 | train_loss 4.2893 | val_loss 4.3399 | val@1 0.056


In [11]:
torch.save(model.state_dict(), '../draftnet_roles_only.pt')
import json
with open('../hero_profile_roles.json', 'w') as f:
    json.dump({str(k):v.tolist() for k,v in hero_profile.items()},f,indent=2)


In [4]:
heroes_raw = list(client.dota.heroes.find({}, {'_id': 0, 'id': 1, 'localized_name': 1}))
name2id = {h['localized_name']: h['id'] for h in heroes_raw}
id2name = {h['id']: h['localized_name'] for h in heroes_raw}

radiant_picks = [
    name2id["Terrorblade"],
    name2id["Ember Spirit"],
    name2id["Magnus"],
    name2id["Oracle"],
]
dire_picks = [
    name2id["Morphling"],
    name2id["Puck"],
    name2id["Beastmaster"],
    name2id["Snapfire"],
]


In [11]:
MODEL_PATH = "../draftnet_roles_only.pt"

def load_model():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    mdl = DraftNet(INPUT_DIM).to(device)
    state = torch.load(MODEL_PATH, map_location="cpu")
    mdl.load_state_dict(state)
    mdl.eval()
    print(f"[LOADED] DraftNet | input_dim={INPUT_DIM}, heroes={HERO_COUNT}")
    return mdl

model = load_model()

[LOADED] DraftNet | input_dim=316, heroes=146


C:\Users\taras\AppData\Local\Temp\ipykernel_29528\2584792098.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(MODEL_PATH, map_location="cpu")


In [12]:
def role_counts(picks):
    if not picks:
        return 0, 0
    carr = int(sum(role_matrix[h, ROLE_IDX['Carry']]   for h in picks))
    supp = int(sum(role_matrix[h, ROLE_IDX['Support']] for h in picks))
    return carr, supp

def build_x(my, opp):
    n_carry, n_support = role_counts(my)
    x = np.zeros(2 * HERO_COUNT + 2 * PROFILE_DIM + 2, dtype=np.float32)

    if my:
        x[np.asarray(my, dtype=np.int64)] = 1.
    if opp:
        x[HERO_COUNT + np.asarray(opp, dtype=np.int64)] = 1.

    x[2*HERO_COUNT : 2*HERO_COUNT + PROFILE_DIM]               = agg_profile(my)
    x[2*HERO_COUNT + PROFILE_DIM : 2*HERO_COUNT + 2*PROFILE_DIM] = agg_profile(opp)

    x[-2] = n_carry
    x[-1] = n_support
    return x

def recommend(my_picks, opp_picks, topk=5):
    x_np   = build_x(my_picks, opp_picks)
    mask_np = np.zeros(HERO_COUNT, dtype=bool)
    mask_np[my_picks + opp_picks] = True

    with torch.no_grad(), torch.cuda.amp.autocast():
        x    = torch.from_numpy(x_np).unsqueeze(0).to(device)
        mask = torch.from_numpy(mask_np).unsqueeze(0).to(device)
        logits = model(x, mask)
        probs  = torch.softmax(logits, dim=1)[0].cpu().numpy()

    top_ids = probs.argsort()[-topk:][::-1]
    return [(id2name[i], float(probs[i])) for i in top_ids]



In [13]:
top = recommend(my_picks=radiant_picks, opp_picks=dire_picks, topk=5)
for name, prob in top:
    print(f"{name:<15} {prob:.3f}")

C:\Users\taras\AppData\Local\Temp\ipykernel_29528\3537327588.py:29: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.no_grad(), torch.cuda.amp.autocast():


Earthshaker     0.042
Leshrac         0.031
Axe             0.028
Huskar          0.026
Tiny            0.026
